## 그룹쿼리 ( COUNT, SUM )
```sql
SELECT productLine, COUNT(*) AS productCount, SUM(quantity*price)
FROM products
GROUP BY productLine;
```

## ERD 프로그램
- Aquerty : 테이블 제한(무료 5개)이 있지만 사용 방법이 쉬고 UI가 간편
- MySQL Workbench : MySQL 데이터베이스용 ERD를 설계, 생성 및 관리하는 통합 도구
- Draw.io(>diagrams.net) : 무료 온라인 다이어그램 도구로, 기본적인 ERD를 쉽게 구현

### MySQL Workbernch ERD
> EDR 다이어그램 생성프로그램<br>
> File > new model > Add Diagram

> 만든 EDR을 테이블 생성 쿼리문으로 전환<br>
> File > Export > Forward Engineer SQL Script > Continue > Continue

> 기존 DB를 EDR로 변환<br>
> Database > Reverse Engineer > Continue > Continue > 스키마 선택 > C..



## Python - MySQL 연동
#### mysql-connector-python
제공자 : Oracle (MySQL 공식)<br>
설치 명령어 : ```pip install mysql-connector-python```
- pymysql보다 약간 더 빠르고 기능이 많고 안정적이다.
- MySQL의 최신 기능을 pymysql보다 더 빠르게 지원한다.
> 공식 드라이버를 선호하고 안정성이 중요하면 사용
#### PyMySQL
제공자 : 오픈소스 커뮤니티 (비공식이지만 널리 사용, MariaDB를 쓴다면 호환성이 더 좋다는 의견이 존재)<br>
설치 명령어 : ```pip install pymysql```
- C로 구현된 부분이 있는 mysql-connector-python와 달리 순수 Python이다.
- mysql-connector-python보다 가볍고 설정이 쉽다.
> 가볍고 간단하게 쓰고 싶으면 사용

## mysql-connector-python
## Faker

In [ ]:
# pip install faker : 가짜 정보 생성
import mysql.connector
from faker import Faker
import random # 파이썬 기본 모듈

# (1) MYSQL 연결
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='0070',
    database='study'
)

# (2) 쿼리 실행하기
cursor = db_connection.cursor()     # SQL 쿼리를 보내는 객체 생성
sql = "INSERT INTO users(username, email) VALUES(%s, %s)"
values = ('다은', 'daeun@naver.com')
cursor.execute(sql, values)


cursor.execute("SELECT user_id FROM users")
valid_user_id = [row[0] for row in cursor.fetchall()]

# (3) MYSQL 연결 끊기
db_connection.commit()
cursor.close()
db_connection.close()

In [ ]:
# input
# 100명의 users 더미 데이터 생성
faker = Faker()
for _ in range(100):
    username = faker.user_name()
    email = faker.email()
    sql = "INSERT INTO users(username, email) VALUES(%s, %s)"
    values = (username, email)
    cursor.execute(sql, values)

In [ ]:
#select
# user_id 불러오기
cursor.execute("SELECT user_id FROM users")

# fetch...() 계열 함수로 결과를 꺼내면 cousor의 내부 버퍼에 저장된 값을 꺼낸 후 버퍼가 비워진다
valid_user_id = [row[0] for row in cursor.fetchall()]
"""
1개만 select해도 튜플형태로 값이 나와서 [(1,), (2,), (3,)]로 나오기 때문에 요소를 꺼내야함
-> 리스트 컴프리헨션 사용 ( 기존 리스트나 반복 가능한 객체로부터 짧고 간결하게 새로운 리스트를 만드는 문법 )
리스트형 : user_info = [(row[1], row[2]) for row in rows]
딕셔너리 : user_dicts = [{'user_id': row[0], 'name': row[1]} for row in rows]
"""

In [ ]:
# 100개의 주문 더미 데이터 생성
for _ in range(100):
    user_id = random.choice(valid_user_id)
    product_name = faker.word()
    quantity = random.randint(1, 10)
    try:
        sql = "INSERT INTO orders(user_id, product_name, quantity) VALUES(%s, %s, %s)"
        values = (user_id, product_name, quantity)
        cursor.execute(sql, values)
    except:
        print("오류 발생")
        pass
db_connection.commit()
cursor.close()
db_connection.close()

## PyMySQL
#### **cursorclass**
데이터베이스 쿼리의 결과를 어떻게 반환할지 결정<br>
Default 빼고는 `pymysql.cursors.(커서이름)`

1. **Default Cursor (`pymysql.cursors.Cursor`)**<br>
(기본 커서)결과를 **튜플 형식**으로 반환, 열 이름 정보는 포함하지 않습니다.
2. **DictCursor**<br>
(**추천**)결과를 **딕셔너리 형식**으로 반환, 각 행은 '컬럼을 키 : 데이터를 값'으로 반환<br>
결과를 처리할 때 컬럼으로 데이터에 접근 ➡️ 결과 데이터를 처리할 때 더 직관적이고 편리함
3. **SSCursor**<br>
서버 사이드 커서로, **큰 결과 집합**을 처리할 때 유용(10000건 이상의 큰 데이터 세트),<br>
모든 결과를 한 번에 메모리에 로드하지 않고, 필요할 때마다 서버에서 행을 로드
4. **SSDictCursor**<br>
서버 사이드 딕셔너리 커서, 'SSCursor'의 기능에 딕셔너리 형식의 결과 반환을 추가

### CRUD
```
Create       새 데이터 생성       INSERT INTO
 Read         데이터 조회           SELECT
Update      기존 데이터 수정         UPDATE
Delete      기존 데이터 삭제       DELETE FROM

In [27]:
import pymysql
# (1) MYSQL 연결
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='0070',
                             db='classicmodels',
                             charset='utf8mb4',     # 이모티콘 포함하기
                             cursorclass=pymysql.cursors.DictCursor)

# (2) 쿼리 실행하기
# INSERT INTO
def add_customer():
    cursor = connection.cursor()    # 리소스 관리를 위해 함수 내에서 생성과 삭제
    name='다은'
    familyNmae='김'
    sql=f"insert into customers(customerNumber, customerName, contactLastName) values({1000},'{name}','{familyNmae}')"
    cursor.execute(sql)
    connection.commit()     # 실행문을 실제 DB에 반영 (select만 제외)
    cursor.close()

# SELECT
def get_customer():
    cursor = connection.cursor()
    sql = "SELECT * FROM customers"
    cursor.execute(sql)
    customers = cursor.fetchone()  # fetchall() 전체 데이터
    print('customers :',customers)
    print('id :',customers['customerNumber'])
    print('country :',customers['city'])
    cursor.close()

# UPDATE
def update_customer():
    cursor = connection.cursor()
    newName='newName'
    newLastName='newLastNmae'
    sql = f"update customers set customername='{newName}',contactLastName='{newLastName}' where customerNumber=1000"
    cursor.execute(sql)
    connection.commit()
    cursor.close()

# DELETE FROM
def delete_customer():
    cursor = connection.cursor()
    sql = "DELETE FROM customers WHERE customerNumber=1000"
    cursor.execute(sql)
    connection.commit()
    cursor.close()

# (3) MYSQL 연결 끊기
connection.close()

### 리팩토링
- with 사용
- execute_query 사용

In [ ]:
import pymysql

def execute_query(connection, query, args=None):
    with connection.cursor() as cursor:                 # with블록 안에서만 cursor 유지
        cursor.execute(query, args or ())
        if query.strip().upper().startswith('SELECT'):
            return cursor.fetchall()
        else:
            connection.commit()

def main():
    connection = pymysql.connect(host='localhost',
                                 user='username',
                                 password='password',
                                 db='database_name',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        # SELECT 연산
        result = execute_query(connection, "SELECT * FROM table_name")
        print("SELECT 연산 결과:")
        for row in result:
            print(row)

        # INSERT 연산
        execute_query(connection, "INSERT INTO table_name (column1, column2) VALUES (%s, %s)", ('data1', 'data2'))
        print("INSERT 연산 수행됨.")

        # UPDATE 연산
        execute_query(connection, "UPDATE table_name SET column1=%s WHERE column2=%s", ('new_data', 'criteria'))
        print("UPDATE 연산 수행됨.")

        # DELETE 연산
        execute_query(connection, "DELETE FROM table_name WHERE column_name=%s", ('criteria',))
        print("DELETE 연산 수행됨.")

    finally:
        connection.close()

if __name__ == "__main__":
    main()